### Challenge 27: Recover the key from CBC with IV=Key

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Find a SHA-1 implementation in the language you code in.

<div class="alert alert-block alert-warning">

### Don't cheat. It won't work.

Do not use the SHA-1 implementation your language already provides (for instance, don't use the "Digest" library in Ruby, or call OpenSSL; in Ruby, you'd want a pure-Ruby SHA-1).

</div>

Write a function to authenticate a message under a secret key by using a secret-prefix MAC, which is simply:

```SHA1(key || message)```

Verify that you cannot tamper with the message without breaking the MAC you've produced, and that you can't produce a new MAC without knowing the secret key. 

</div>

Found this implementation from [https://gist.github.com/BenWiederhake/cb60f703840f9e81a84499b39eb361b5](https://gist.github.com/BenWiederhake/cb60f703840f9e81a84499b39eb361b5)

In [23]:
from Crypto import Random

In [15]:
#!/usr/bin/env python3

# Based on https://gist.github.com/bonsaiviking/5639034
# Converted to Python3 by hand.

import struct

def leftrotate(i, n):
    return ((i << n) & 0xffffffff) | (i >> (32 - n))

class SHA1:
    def __init__(self, data=b''):
        self.h = [
                0x67452301,
                0xEFCDAB89,
                0x98BADCFE,
                0x10325476,
                0xC3D2E1F0
                ]
        self.remainder = data
        self.count = 0

    def _add_chunk(self, chunk):
        self.count += 1
        w = list( struct.unpack(">16I", chunk) + (None,) * (80-16) )
        for i in range(16, 80):
            n = w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]
            w[i] = leftrotate(n, 1)
        a,b,c,d,e = self.h
        for i in range(80):
            f = None
            k = None
            if i < 20:
                f = (b & c) ^ (~b & d)
                k = 0x5A827999
            elif i < 40:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif i < 60:
                f = (b & c) ^ (b & d) ^ (c & d)
                k = 0x8F1BBCDC
            else:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = (leftrotate(a,5) + f + e + k + w[i]) % 2**32
            e = d
            d = c
            c = leftrotate(b, 30)
            b = a
            a = temp
        self.h[0] = (self.h[0] + a) % 2**32
        self.h[1] = (self.h[1] + b) % 2**32
        self.h[2] = (self.h[2] + c) % 2**32
        self.h[3] = (self.h[3] + d) % 2**32
        self.h[4] = (self.h[4] + e) % 2**32

    def add(self, data):
        message = self.remainder + data
        r = len(message) % 64
        if r != 0:
            self.remainder = message[-r:]
        else:
            self.remainder = b''
        for chunk in range(0, len(message)-r, 64):
            self._add_chunk( message[chunk:chunk+64] )
        return self

    def finish(self):
        l = len(self.remainder) + 64 * self.count
        self.add( b'\x80' + b'\x00' * ((55 - l) % 64) + struct.pack(">Q", l * 8) )
        h = tuple(x for x in self.h)
        self.__init__()
        return struct.pack(">5I", *h)

Write a function to authenticate a message under a secret key by using a secret-prefix MAC, which is simply:

```SHA1(key || message)```

Verify that you cannot tamper with the message without breaking the MAC you've produced, and that you can't produce a new MAC without knowing the secret key. 


In [29]:
MAC_key = Random.get_random_bytes(32)

def make_MAC(key, message):
    
    my_sha = SHA1(MAC_key + bytes(message))
    MAC = my_sha.finish()
    
    return(MAC)

In [30]:
message = b'Good message'
message_mac = make_MAC(MAC_key, message)

bad_message = b'Good massage'
bad_msg_mac = make_MAC(MAC_key, bad_message)

print(message_mac)
print(bad_msg_mac)
print(f"Match = {message_mac == bad_msg_mac}")

b'\xf7{vz\x84\x97\x1fk0\xa4\x9b\xc5\'\xdf"~o\xab\xa2x'
b'}|\xa4\xb4z\xc4\x8c\xd1\xac)e!\xc3\xde<P,\xde\x1ek'
Match = False


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)